### Importing the Libraries

In [5]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [6]:
tf.__version__ 

'2.11.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [19]:
training_set = train_datagen.flow_from_directory('D:/dataSet/trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 12845 images belonging to 27 classes.


In [20]:
test_set = test_datagen.flow_from_directory('D:/dataSet/testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 4268 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [22]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [23]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [24]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [25]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [26]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [27]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [28]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [29]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)              

In [31]:
classifier.fit(training_set,
                  epochs = 5,
                  validation_data = test_set)

Epoch 1/5
1285/1285 [==============================] - 290s 226ms/step - loss: 0.8282 - accuracy: 0.7197 - val_loss: 0.1357 - val_accuracy: 0.9613
Epoch 2/5
1285/1285 [==============================] - 273s 212ms/step - loss: 0.5591 - accuracy: 0.8105 - val_loss: 0.0555 - val_accuracy: 0.9838
Epoch 3/5
1285/1285 [==============================] - 279s 217ms/step - loss: 0.4309 - accuracy: 0.8561 - val_loss: 0.0262 - val_accuracy: 0.9951
Epoch 4/5
1285/1285 [==============================] - 257s 200ms/step - loss: 0.3313 - accuracy: 0.8905 - val_loss: 0.0221 - val_accuracy: 0.9930
Epoch 5/5
1285/1285 [==============================] - 212s 165ms/step - loss: 0.2790 - accuracy: 0.9063 - val_loss: 0.0118 - val_accuracy: 0.9970


#### Saving the Model

In [32]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved
